# Imports

In [1]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Bidirectional, LSTM, GRU, TimeDistributed, Activation, Flatten, Embedding
from keras.optimizers import Adam



/Users/jmangia/anaconda2/envs/SwiftNLC/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Functions

In [2]:
# Prepare Glove File
def readGloveFile(gloveFile):
    with open(gloveFile, 'r') as f:
        wordToGlove = {}  # map from a token (word) to a Glove embedding vector
        wordToIndex = {}  # map from a token to an index
        indexToWord = {}  # map from an index to a token 

        for line in f:
            record = line.strip().split()
            token = record[0] # take the token (word) from the text line
            wordToGlove[token] = np.array(record[1:], dtype=np.float64) # associate the Glove embedding vector to a that token (word)

        tokens = sorted(wordToGlove.keys())
        for idx, tok in enumerate(tokens):
            kerasIdx = idx + 1  # 0 is reserved for masking in Keras (see above)
            wordToIndex[tok] = kerasIdx # associate an index to a token (word)
            indexToWord[kerasIdx] = tok # associate a word to a token (word). Note: inverse of dictionary above

    return wordToIndex, indexToWord, wordToGlove

In [3]:
# Create Pretrained Keras Embedding Layer
def createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, inputLength, isTrainable):
    vocabLen = len(wordToIndex) + 1  # adding 1 to account for masking
    embDim = next(iter(wordToGlove.values())).shape[0]  # works with any glove dimensions (e.g. 50)

    embeddingMatrix = np.zeros((vocabLen, embDim))  # initialize with zeros
    for word, index in wordToIndex.items():
        embeddingMatrix[index, :] = wordToGlove[word] # create embedding: word index to Glove word embedding

    embeddingLayer = Embedding(vocabLen, embDim, weights=[embeddingMatrix], input_length=inputLength, trainable=isTrainable)
    return embeddingLayer, embDim

In [4]:
# Embedding
def getEncodedDocs(docs):
    encoded_docs = []

    tokenizer = RegexpTokenizer(r'\w+')
    for doc in docs:
        encoded_doc = []
        for word in tokenizer.tokenize(doc.lower()):
            index = wordToIndex[word]
            if index is not None:
                encoded_doc.append(index)
            else:
                encoded_doc.append(0)
        encoded_docs.append(encoded_doc)

    return encoded_docs

# Import GloVe Pretrained dataset and create Embedding Layer

In [5]:
max_length = 20
wordToIndex, indexToWord, wordToGlove = readGloveFile("glove/glove.6B.100d.txt")
pretrainedEmbeddingLayer, embDim = createPretrainedEmbeddingLayer(wordToGlove, wordToIndex, max_length, False)

# Import intents file

In [6]:
import json
with open('PharmacyDataset.json') as json_data:
    intents = json.load(json_data)

# Padding, Encoding and Preparing final X and Y data for Training

In [7]:
classes = []
encodedUtterances = []

# loop through each sentence in our intents utterances
for intent in intents['intents']:
    classes.append(intent['intent'])
    encoded_docs = getEncodedDocs(intent['utterances'])
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    encodedUtterances.append(padded_docs)


In [8]:
currentClass = 0
train_x = []
train_y = []

for intent in classes:
    y = [0] * len(classes)
    y[currentClass] = 1

    for vector in encodedUtterances[currentClass]:
        train_x.append(vector)
        train_y.append(y)

    currentClass += 1

vectorSize = len(train_x[0])

train_X = np.array(train_x)
train_Y = np.array(train_y)

print(classes, "classes")
print(vectorSize, "vector size")
print(len(train_x), len(train_x[0]), "x")
print(len(train_y), len(train_y[0]), "y")
print(train_X.shape)
print(train_Y.shape)

([u'greeting', u'goodbye', u'thanks', u'options', u'adverse_drug', u'blood_pressure', u'blood_pressure_search', u'pharmacy_search'], 'classes')
(20, 'vector size')
(26, 20, 'x')
(26, 8, 'y')
(26, 20)
(26, 8)


# Prepare and Compile Keras / TensorFlow model

In [9]:
#sequence_input = Input(shape=(None, len(train_x[0])), dtype='float')
#bidiGru = Bidirectional(GRU(100))(sequence_input)
#preds = Dense(len(train_y[0]), activation='softmax')(bidiGru)
#model = Model(sequence_input, preds)
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

model = Sequential([
                    pretrainedEmbeddingLayer,
                    GRU(embDim, batch_size=1, input_shape=(None, embDim), return_sequences=True),
                    TimeDistributed(Dense(64)),
                    Activation('relu'),
                    TimeDistributed(Dense(32)),
                    Activation('relu'),
                    Flatten(),
                    Dense(len(train_y[0]), activation='softmax')
                   ])

print("model fitting - Bidirectional GRU")
model.summary()


model fitting - Bidirectional GRU
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           40000100  
_________________________________________________________________
gru_1 (GRU)                  (None, 20, 100)           60300     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 64)            6464      
_________________________________________________________________
activation_1 (Activation)    (None, 20, 64)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 32)            2080      
_________________________________________________________________
activation_2 (Activation)    (None, 20, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 640)  

In [10]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train Model

In [11]:
model.fit(train_X, train_Y, epochs=50)

Epoch 1/50
26/26 [==============================] - 0s 15ms/step - loss: 2.1176 - acc: 0.0385
Epoch 2/50
26/26 [==============================] - 0s 738us/step - loss: 2.0691 - acc: 0.1538
Epoch 3/50
26/26 [==============================] - 0s 699us/step - loss: 2.0279 - acc: 0.2308
Epoch 4/50
26/26 [==============================] - 0s 635us/step - loss: 1.9894 - acc: 0.3077
Epoch 5/50
26/26 [==============================] - 0s 682us/step - loss: 1.9519 - acc: 0.3846
Epoch 6/50
26/26 [==============================] - 0s 624us/step - loss: 1.9143 - acc: 0.4231
Epoch 7/50
26/26 [==============================] - 0s 658us/step - loss: 1.8752 - acc: 0.4615
Epoch 8/50
26/26 [==============================] - 0s 639us/step - loss: 1.8334 - acc: 0.5000
Epoch 9/50
26/26 [==============================] - 0s 681us/step - loss: 1.7891 - acc: 0.5000
Epoch 10/50
26/26 [==============================] - 0s 644us/step - loss: 1.7423 - acc: 0.5000
Epoch 11/50
26/26 [==============================]

# Save Model

In [12]:
model.save('SwiftNLCGloveRNN.h5')

# Test Model

In [20]:
# evaluate the model
loss, accuracy = model.evaluate(train_X, train_Y, verbose=0)
print('Loss: %f ' % (loss*100))
print('Accuracy: %f ' % (accuracy*100))

Loss: 0.724069 
Accuracy: 100.000000 


In [46]:
print(intents)

#test = np.array([[178126, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
test_docs = getEncodedDocs(["Display blood values for patient"])
padded_test_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')

print(padded_test_docs)

y_pred = model.predict(padded_test_docs)
print(y_pred)

max_value = max(y_pred[0])
print(max_value)

max_index = y_pred[0].tolist().index(max_value)
print(max_index)

print(intents['intents'][max_index]['intent'])

{u'intents': [{u'intent': u'greeting', u'utterances': [u'Hi', u'How are you', u'Is anyone there?', u'Hello', u'Good day']}, {u'intent': u'goodbye', u'utterances': [u'Bye', u'See you later', u'Goodbye']}, {u'intent': u'thanks', u'utterances': [u'Thanks', u'Thank you', u"That's helpful"]}, {u'intent': u'options', u'utterances': [u'How you could help me?', u'What you can do?', u'What help you provide?']}, {u'intent': u'adverse_drug', u'utterances': [u'How to check Adverse drug reaction?', u'List all drugs suitable for patient with adverse reaction', u'Which drugs dont have adverse reaction?']}, {u'intent': u'blood_pressure', u'utterances': [u'Open blood pressure module', u'I want to log blood pressure results', u'Blood pressure data management']}, {u'intent': u'blood_pressure_search', u'utterances': [u'I want to search for blood pressure result history', u'Show blood pressure results for patient', u'Find blood pressure results by ID']}, {u'intent': u'pharmacy_search', u'utterances': [u'Fi

# Export Model using CoreML Tools

In [13]:
import coremltools
coreml_model = coremltools.converters.keras.convert(model, input_names="vectors", output_names="entities")
coreml_model

0 : embedding_1_input, <keras.engine.topology.InputLayer object at 0x1464cb050>
1 : embedding_1, <keras.layers.embeddings.Embedding object at 0x126ef5550>
2 : gru_1, <keras.layers.recurrent.GRU object at 0x12cadfd10>
3 : time_distributed_1, <keras.layers.wrappers.TimeDistributed object at 0x12cadff90>
4 : activation_1, <keras.layers.core.Activation object at 0x12cadffd0>
5 : time_distributed_2, <keras.layers.wrappers.TimeDistributed object at 0x11029fed0>
6 : activation_2, <keras.layers.core.Activation object at 0x11029fb90>
7 : flatten_1, <keras.layers.core.Flatten object at 0x11029fa10>
8 : dense_3, <keras.layers.core.Dense object at 0x11029fd50>
9 : dense_3__activation__, <keras.layers.core.Activation object at 0x128c4a7d0>


input {
  name: "vectors"
  type {
    multiArrayType {
      shape: 1
      dataType: DOUBLE
    }
  }
}
input {
  name: "gru_1_h_in"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
    isOptional: true
  }
}
output {
  name: "entities"
  type {
    multiArrayType {
      shape: 8
      dataType: DOUBLE
    }
  }
}
output {
  name: "gru_1_h_out"
  type {
    multiArrayType {
      shape: 100
      dataType: DOUBLE
    }
  }
}

In [14]:
coreml_model.save('SwiftNLCGloveRNN.mlmodel')